# Homework #1: Keyword extraction
Prepare dataset and gold standard, use methods for extraction, and evaluate.

## Imports

In [ ]:
# pip3 install bs4
# pip3 install pandas
# pip3 install stanza
# pip3 install nltk
# pip3 install python-rake
# pip3 install summa
# pip3 install sentence-transformers
# pip3 install numpy
# pip3 install scikit-learn

In [36]:
import random
import re
import requests
import statistics
import time
from typing import Callable
from bs4 import BeautifulSoup
import nltk
import numpy as np
import pandas as pd
import RAKE
from scipy.sparse._csr import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import stanza
import summa

In [ ]:
# nltk.download('popular')

In [2]:
rus_sw = set(nltk.corpus.stopwords.words('russian'))

## Dataset
Let`s get the articles from [habr.com](https://habr.com/ru/all/) that are related with NLP.

In [ ]:
def parse_site(url: str, ua: str) -> BeautifulSoup:
    '''
    Parse web-site with BeautifulSoup.
    '''

    res = requests.get(url, headers={'User-Agent': ua})
    page = res.text
    soup = BeautifulSoup(page, 'html.parser')

    return soup

In [ ]:
uas = [
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:91.0) Gecko/20100101 Firefox/91.0',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.43 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36 OPR/77.0.4054.277'
]

* Parse site with links

In [ ]:
SEARCH_NLP_RESULTS = 'https://habr.com/ru/search/?q=NLP&target_type=posts&order=date'

search_nlp_results = parse_site(SEARCH_NLP_RESULTS, random.choice(uas))

In [ ]:
articles_list = search_nlp_results.find(
    'div',
    {
        'data-test-id': 'articles-list',
        'class': 'tm-articles-list'
    },
    )
articles_boxes = articles_list.find_all(
    'article'
)
print('Статей на первой странице:', len(articles_boxes))

nlp_links = []
for box in articles_boxes[:10]:
    title = box.find(
        'a',
        {
            'class': 'tm-article-snippet__title-link'
        }
    )
    link = title.attrs['href']
    nlp_links.append('https://habr.com' + link)

nlp_links

* Parse articles with 5 or more tags (the value obtained empirically)

Keywords in this articles are presented in the tags section. Users tag their articles with some words or n-grams by which we can search related articles on the resource.

In [ ]:
articles_data = {
    'link': [],
    'original tags': [],
    'text': []
}

for link in nlp_links:

    time.sleep(3)

    article = parse_site(link, random.choice(uas))

    tags_a = article.find_all(
        'a',
        {
            'class': 'tm-tags-list__link'
        }
    )

    if len(tags_a) >= 5:

        articles_data['link'].append(link)

        # tags
        tags = []
        for tag_a in tags_a:
            tags.append(tag_a.text)
        articles_data['original tags'].append(', '.join(tags))

        # text
        text = []
        full_text = article.find(
        'div',
        {
            'xmlns': 'http://www.w3.org/1999/xhtml'
        })
        try:
            for elem in full_text:
                text.append(elem.text)
            text = '\n'.join(text)
        except AttributeError:
            text = full_text.text
        articles_data['text'].append(text)

In [ ]:
articles_data['link'][1]

In [ ]:
articles_df = pd.DataFrame(articles_data)

articles_df.head(7)

In [ ]:
articles_df['link'][0]

## Gold standard
Update dataset for our keywords and gold standard

* Save dataset

*Suddenly, a new article appeared...*

In [ ]:
articles_df = articles_df[1:6]

articles_df

* Check length of the data

In [ ]:
texts = articles_df['text'].to_list()

In [ ]:
length = 0
for text in texts:
    length += len(text.split())

In [ ]:
length

* New dataset

__Keywords selection methodology__:
* New keywords are selected manually. To do that we must read the whole article.
* Keywords in this case is a list of expressions that can help to search this article in the Web. In the same time the list can be perceived like summary of the text (due to the specific of the texts).
* These keywords must be presented in the text of the article.
* List contains synonymes (ex. *NLP* and *обработка естественного языка*). 
* Keywords consist from words and collocations (n-grams).
* Term can appear 1 time in the text but in fact it represents the important information about the text.
* Collocations are not directly extracted from text (not 'as is'). Head words were given to the nominative, but collocates were agreed in number and case. There are not lemmas but collocations.
* Keywords are in Russian and English due to the specific of the texts (*technologies*, *informational technologies*, *programming*, etc.)

*See more about markup and gold standard in the comments below*

In [ ]:
with open('new_keywords.txt', 'r', encoding='utf-8') as f:
    new_tags = f.read().split('\n')[0:5]

In [ ]:
articles_df['new tags'] = new_tags

In [ ]:
articles_df.head()

__Comparative analysis of the keyword lists:__
* Original tags contain tags that are not indicated in the text directly but they are related with the text and can improve informational retrieval. Our keywords only contain expressions from the texts.
* Original tags are mostly in the lower case. In the list of the new tags we can see all cases.

__How to prepare the gold standard?__
* Take our keywords and add it to orginal tags. 
* Remove words and expressions that are not presented in the original text.
* Do all these actions manually.

__Questions?__
* __Why manually?__ This is gold standard, and the gold set must be close to ideal. Humans are native speakers of the language and could resolve many language problems better then machine.
* __Why we remove the keywords that are not in the text?__ RAKE and TextRank are based on the words of the one particular text. That is why we can not include in the gold set absent expressions.
* __Why we keep the terms on English?__ Because we can rank the expressions regardless the language. BERT-Encoder can be based on the multilingual set.

In [ ]:
articles_df['original tags']

In [ ]:
original_tags = [[expression.lower() for expression in text] for text in articles_df['original tags'].str.split(', ')]
new_tags = [[expression.lower() for expression in text] for text in articles_df['new tags'].str.split(', ')]

print(original_tags)

In [ ]:
gold_tags = []
for orig, new in zip(original_tags, new_tags):
    gold = set(orig + new)
    gold_tags.append(', '.join(gold))

In [ ]:
gold_tags[1]

In [ ]:
articles_df['gold tags'] = gold_tags

* Lemmatized gold tags

In [ ]:
with open('gold lemmatized.txt', 'r', encoding='utf-8') as fl:
    gold_lemmatized = fl.read().split('\n\n')

In [ ]:
articles_df['lemmatized gold tags'] = gold_lemmatized

* Lemmatized gold tags with POS-tags

In [ ]:
with open('gold lemmatized pos.txt', 'r', encoding='utf-8') as flp:
    gold_lemmatized_pos = flp.read().split('\n\n')

In [ ]:
articles_df['lemmatized pos gold tags'] = gold_lemmatized_pos

* Length of the gold standard

In [ ]:
articles_df['keywords length'] = articles_df['gold tags'].str.split(', ').str.len()

* Save keywords that correspond to patterns

Most frequent patterns are following:
* NOUN (*технология*, *хакатон*)
* PROPN (*SberDevices*, *ruBERT*)
* ADJ + NOUN (*нейронная сеть*, *машинное обучение*)
* NOUN + NOUN (*распознавание речи*, *обработка текста*)

In [66]:
POS_TAGS = [['NOUN'], ['PROPN'], ['ADJ', 'NOUN'], ['NOUN', 'NOUN']]

In [20]:
pattern_keywords = []
for text in gold_lemmatized_pos:
    new_text = []
    for keyword in text.split(', '):
        pos_list = []
        for word in keyword.split():
            pos = word.split('_')[1]
            pos_list.append(pos)
        if pos_list in POS_TAGS:
            new_text.append(keyword)
    new_text = ', '.join(new_text)
    pattern_keywords.append(new_text)

In [21]:
articles_df['patterns gold tags'] = pattern_keywords

In [22]:
articles_df

,link,original tags,text,new tags,gold tags,keywords length,lemmatized gold tags,lemmatized pos gold tags,patterns gold tags
0,https://habr.com/ru/company/just_ai/news/t/698...,"искусственный интеллект, голосовые интерфейсы,...",2 декабря в Москве в онлайн- и офлайн-формате ...,"Conversations, конференция, NLP, диалоговые пл...","conversations, deeppavlov, sberdevices, yandex...",23,"conversations, deeppavlov, sberdevices, yandex...","conversations_PROPN, deeppavlov_PROPN, sberdev...","conversations_PROPN, deeppavlov_PROPN, sberdev..."
1,https://habr.com/ru/company/first/blog/699380/,"open source, ai, ml, инструменты разработчика,...",\nОт создания дипфейков до обработки естествен...,"технологии, опенсорс, машинное обучение, глубо...","технологии, нейронная сеть, приложение, ai, гл...",28,"технология, нейронный сеть, приложение, ai, гл...","технология_NOUN, нейронный_ADJ сеть_NOUN, прил...","технология_NOUN, нейронный_ADJ сеть_NOUN, прил..."
2,https://habr.com/ru/company/alfa/blog/698660/,"хакатон, граф, машинное обучение, нейросети, с...",В сентябре 2022 проходил хакатон «Машинное обу...,"хакатон, графы, VK, ВК, Альфа-Банк, машинное о...","1 место, хакатон, графы, альфа-банк, обработка...",13,"1 место, хакатон, граф, альфа-банк, обработка ...","1_NUM место_NOUN, хакатон_NOUN, граф_NOUN, аль...","хакатон_NOUN, граф_NOUN, альфа-банк_NOUN, обра..."
3,https://habr.com/ru/company/amvera/blog/699058/,"генерирование стихов, генерация текста, создан...","Мне нравится концепция, согласно которой речь ...","обработка естественного языка, NLP, нейронные ...","rubert, сгенерировать стихотворения, lstm, ней...",12,"rubert, сгенерировать стихотворение, lstm, ней...","rubert_PROPN, сгенерировать_VERB стихотворение...","rubert_PROPN, lstm_PROPN, нейронный_ADJ сеть_N..."
4,https://habr.com/ru/company/unidata/blog/698268/,"entity resolution, data matching, record linka...",Всем привет!\nМы хотим рассказать немного об e...,"entity resolution, дедупликация, повторы, дубл...","дубликаты, нейросети, entity resolution, data ...",16,"дубликат, нейросеть, entity resolution, data m...","дубликат_NOUN, нейросеть_NOUN, entity_NOUN res...","дубликат_NOUN, нейросеть_NOUN, entity_NOUN res..."


In [24]:
articles_df.to_csv('articles_df.tsv', sep='\t')

*It seems that only our tags remained.*

## Extract keywords
Extract keywords from the parsed texts with RAKE, TextRank and BERT-Encoder

__Preprocessing:__

In [25]:
texts = articles_df['text'].to_list()

In [26]:
cleaned_texts = []
for text in texts:
    text = re.sub('\s+', ' ', text)
    text = re.sub('•', '', text)
    cleaned_texts.append(text)

In [27]:
cleaned_texts[0]

"2 декабря в Москве в онлайн- и офлайн-формате состоится Conversations – ежегодная конференция по разговорному AI для разработчиков и бизнеса. Про NLP-сервисы, диалоговые платформы и фреймворки, синтез и распознавание речи, UX и проектирование разговорных интерфейсов, генеративные модели и не только расскажут KODE, MTS AI, Альфа-Банк, Сбер, Yandex Cloud, DeepPavlov и другие эксперты. В нашем анонсе – особо интригующие спойлеры и промокод на скидку. Участников конференции ждет 2 потока докладов – Business и Technology. С полной программой можно познакомиться на сайте Conversations, ну а мы предлагаем погрузиться именно в Technology Track. Секция «Good morning, ML!» Что делать, если исходящий звонок колл-центра попадает на голосовую почту или умного ассистента? О технологиях ML для создания модели идентификации голосовой почты и виртуального помощника расскажет Артем Бондарь, Voximplant.Мурат Апишев из Just AI раскроет детали работы над индустриальной NLP-платформой: как организовали упр

In [28]:
nlp = stanza.Pipeline('ru')

2022-11-20 01:48:18 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2022-11-20 01:48:18 INFO: Use device: cpu
2022-11-20 01:48:18 INFO: Loading: tokenize
2022-11-20 01:48:19 INFO: Loading: pos
2022-11-20 01:48:19 INFO: Loading: lemma
2022-11-20 01:48:19 INFO: Loading: depparse
2022-11-20 01:48:20 INFO: Loading: ner
2022-11-20 01:48:21 INFO: Done loading processors!


In [29]:
def stanza_nlp(text: str):
    '''
    Get lemmatized text with stanza.
    '''

    text_lemmas = []
    text_lemmas_pos = []

    doc = nlp(text)
    for sent in doc.sentences:
        for word in sent.words:
            if word.pos != 'PUNCT' and word.lemma not in rus_sw:
                text_lemmas.append(word.lemma.lower())
                text_lemmas_pos.append(f'{word.lemma.lower()}_{word.pos}')

    return ' '.join(text_lemmas), ' '.join(text_lemmas_pos)

In [30]:
lemmas = [[], []]
for text in cleaned_texts:
    processed = stanza_nlp(text)
    for container, lemmatized in zip(lemmas, processed):
        container.append(lemmatized)

In [31]:
lemmas[0][0]  # lemmas

"2 декабрь москва онлайн офлайн формат состояться conversations ежегодный конференция разговорный ai разработчик бизнес nlp сервис диалоговый платформа фреймворка синтез распознавание речь ux проектирование разговорный интерфейс генеративный модель рассказать kode mts ai альфа-банк сбер yandex cloud deeppavlov эксперт наш анонсе особо интриговать спойлера промокод скидка участник конференция ждать 2 поток доклад business technology полный программа знакомиться сайт conversations предлагать погрузиться именно technology track секция good morning ml делать исходить звонок колл-центр попадать голосовой почта умный ассистент технология ml создание модель идентификация голосовой почта виртуальный помощник рассказать артем бондарь voximplant.мурат апишев just ai раскроить деталь работа индустриальный nlp платформа организовывать управление nlp сервис модель стандартизировать встраивание собственный open-source решение выучить свой управлять парафраз помощь пользователь делать модель который 

In [32]:
lemmas[1][0]  # lemmas with pos

"2_NUM декабрь_NOUN москва_PROPN онлайн_NOUN офлайн_NOUN формат_NOUN состояться_VERB conversations_PROPN ежегодный_ADJ конференция_NOUN разговорный_ADJ ai_PROPN разработчик_NOUN бизнес_NOUN nlp_PROPN сервис_NOUN диалоговый_ADJ платформа_NOUN фреймворка_NOUN синтез_NOUN распознавание_NOUN речь_NOUN ux_PROPN проектирование_NOUN разговорный_ADJ интерфейс_NOUN генеративный_ADJ модель_NOUN рассказать_VERB kode_PROPN mts_PROPN ai_PROPN альфа-банк_PROPN сбер_PROPN yandex_PROPN cloud_PROPN deeppavlov_PROPN эксперт_NOUN наш_DET анонсе_NOUN особо_ADV интриговать_VERB спойлера_NOUN промокод_NOUN скидка_NOUN участник_NOUN конференция_NOUN ждать_VERB 2_NUM поток_NOUN доклад_NOUN business_PROPN technology_PROPN полный_ADJ программа_NOUN знакомиться_VERB сайт_NOUN conversations_PROPN предлагать_VERB погрузиться_VERB именно_PART technology_PROPN track_PROPN секция_NOUN good_PROPN morning_PROPN ml_PROPN делать_VERB исходить_VERB звонок_NOUN колл-центр_NOUN попадать_VERB голосовой_ADJ почта_NOUN умный_A

In [65]:
POS_TAGS_LOWER = [['noun'], ['propn'], ['adj', 'noun'], ['noun', 'noun']]

In [63]:
def get_patterns(text: str) -> str:
    '''
    Get patterns from keywords text.
    '''

    if text == '':

        return ''

    new_text = []

    for keyword in text.split(', '):
        pos_list = []
        keywords = keyword.split()
        for word in keywords:
            try:
                pos = word.split('_')[1]
                pos_list.append(pos)
            except IndexError:
                pass
            if len(pos_list) == len(keywords):
                if pos_list in POS_TAGS_LOWER or pos_list in POS_TAGS:
                    new_text.append(keyword)

    new_text = ', '.join(new_text)

    return new_text

* __RAKE__

In [37]:
Rake = RAKE.Rake(list(rus_sw))

*Already without stopwords from the set*

* On the raw corpus

In [34]:
def extract_from_raw(func: Callable, texts: list, mode='summa', **kwargs) -> list:
    '''
    Keyword extraction for all of the texts in the corpora given
    with the extractors on the whole texts.
    '''

    keywords = []

    for text in texts:
        if mode == 'rake':
            coll_list = ', '.join([coll[0] for coll in func(text, **kwargs)])
        else:
            coll_list = ', '.join(func(text, **kwargs).split('\n'))
        keywords.append(coll_list)

    return keywords

Max: trigrams

In [38]:
rake_raw_3 = extract_from_raw(Rake.run, cleaned_texts, mode='rake', maxWords=3)

rake_raw_3[0]

'организовали управление nlp-сервисами, классические блок-схемы устарели, офлайн-формате состоится conversations, почему древовидные схемы, проектированию блок-схем, проектирование разговорных интерфейсов, особо интригующие спойлеры, предлагаем погрузиться именно, индустриальной nlp-платформой, стандартизировали встраивание собственных, open-source решений, основе генеративных моделей, улучшение качества ответов, создать целевую инфраструктуру, какую аналитику учесть, планировании виртуального помощника, секция «good morning, личности» никита блинков, секция «set tool, современные инструменты работы, разработке личности ассистента, разработке личности, nlp-сервисы, сайте conversations, секция «bot, умного ассистента, ежегодная конференция, разговорному ai, диалоговые платформы, распознавание речи, генеративные модели, mts ai, альфа-банк, yandex cloud, другие эксперты, нашем анонсе, полной программой, голосовую почту, мурат апишев, помощи пользователям, сделать модель, которая понимает,

In [39]:
rake_lemmas_3 = extract_from_raw(Rake.run, lemmas[0], mode='rake', maxWords=3)

rake_lemmas_3[0]

''

*:(*

In [40]:
rake_lemmas_pos_3 = extract_from_raw(Rake.run, lemmas[1], mode='rake', maxWords=3)

rake_lemmas_pos_3[0]

''

In [67]:
rake_lemmas_pos_patt_3 = []
for text in rake_lemmas_pos_3:
    pattern_text = get_patterns(text)
    rake_lemmas_pos_patt_3.append(pattern_text)

In [68]:
rake_lemmas_pos_patt_3[0]

''

Max: bigrams

In [41]:
rake_raw_2 = extract_from_raw(Rake.run, cleaned_texts, mode='rake', maxWords=2)

rake_raw_2[0]

'open-source решений, проектированию блок-схем, индустриальной nlp-платформой, nlp-сервисы, ежегодная конференция, разговорному ai, диалоговые платформы, распознавание речи, генеративные модели, mts ai, альфа-банк, yandex cloud, другие эксперты, нашем анонсе, полной программой, сайте conversations, голосовую почту, умного ассистента, мурат апишев, помощи пользователям, сделать модель, которая понимает, тёти сары, фрау заурих, ответы готовы, антона ермилова, мария тихонова, проведет экскурсию, секция «bot, s developing, группам внутри, отдельных интентов, ангелина большина, галина прохорова, разработке личности, раскроют инсайты, своего исследования, который основан, эволюционном подходе, кириллом богатовым, контроль диалога, помощью pals, данила корнев, андрей татаринов, фреймворком rasa, сделать бота, иерархическим классификатором, опыт трансформации, legacy-систем, станислав милых, ребята научат, читателей хабра, прошлом году, расскажут kode, technology track, технологиях ml, ml-инст

In [42]:
rake_lemmas_2 = extract_from_raw(Rake.run, lemmas[0], mode='rake', maxWords=2)

rake_lemmas_2[0]

''

*:(*

In [43]:
rake_lemmas_pos_2 = extract_from_raw(Rake.run, lemmas[1], mode='rake', maxWords=2)

rake_lemmas_pos_2[0]

''

In [69]:
rake_lemmas_pos_patt_2 = []
for text in rake_lemmas_pos_2:
    pattern_text = get_patterns(text)
    rake_lemmas_pos_patt_2.append(pattern_text)

In [70]:
rake_lemmas_pos_patt_2[0]

''

* __TextRank__

In [44]:
summa_raw = extract_from_raw(summa.keywords.keywords, cleaned_texts, language='russian', additional_stopwords=rus_sw)

summa_raw[0]

'моделями, ai, генеративные модели, генеративных моделей, модель которая, секция, ml, расскажут, инструменты, conversations, который, разговорному, разговорных, личности, nlp диалоговые, диалоговых, голосовую, голосовой, голосового, помощи, помощью, почему, kode, помощника расскажет, свой, своего, ответы, ответов, автоматический, автоматическое, фреймворки, фреймворком, аналитику, ангелина, airtable, году, целевую, речи, погрузиться, подходах, подход, подходят, подходе, подхода, конференция, конференции, deeppavlov, решений, решение, yandex, cloud, ассистента, банке, милых, ребята, инсайты, инсайтами, анонсе, звонок, хабра, интригующие'

In [45]:
summa_lemmas = extract_from_raw(summa.keywords.keywords, lemmas[0], language='russian')

summa_lemmas[0]

'подход, подходить, генеративный модель рассказать kode, ml делать, помощь, конференция разговорный ai, инструмент, секция, работа, conversations, сценарий личность, голосовой, помощник, проектирование, ответ, бот банк, technology, nlp, класс, почему, интент, решение, автоматический, свой, который, бота, блок схема, yandex cloud deeppavlov, инсайт, фреймворкое, диалоговый платформа фреймворка, спикер, ассистент, управление, онлайн офлайн, промокод скидка, мочь, сайт'

In [46]:
summa_lemmas_pos = extract_from_raw(summa.keywords.keywords, lemmas[1], language='russian')

summa_lemmas_pos[0]

'рассказать_verb, модель_noun, ai_propn, _num, подход_noun, ml_propn, личность_noun, помощь_noun, инструмент_noun, секция_noun, проектирование_noun, генеративный_adj, работа_noun, разговорный_adj, делать_verb, conversations_propn, голосовой_adj, помощник_noun, deeppavlov_propn, ответ_noun, nlp_propn, класс_noun, technology_propn, почему_adv, схема_noun, интент_noun, решение_noun, свой_det, автоматический_adj, который_pron, бота_noun, бот_noun, сценарий_noun, конференция_noun, диалоговый_adj, kode_propn, инсайт_noun, фреймворка_noun, фреймворкое_noun, спикер_noun, ассистент_noun, платформа_noun, управление_noun, yandex_propn, cloud_propn, офлайн_noun, мочь_verb, онлайн_noun, промокод_noun, год_noun, сайт_noun, виртуальный_adj'

In [71]:
summa_lemmas_pos_patt = []
for text in summa_lemmas_pos:
    pattern_text = get_patterns(text)
    summa_lemmas_pos_patt.append(pattern_text)

In [72]:
summa_lemmas_pos_patt[0]

'модель_noun, ai_propn, подход_noun, ml_propn, личность_noun, помощь_noun, инструмент_noun, секция_noun, проектирование_noun, работа_noun, conversations_propn, помощник_noun, deeppavlov_propn, ответ_noun, nlp_propn, класс_noun, technology_propn, схема_noun, интент_noun, решение_noun, бота_noun, бот_noun, сценарий_noun, конференция_noun, kode_propn, инсайт_noun, фреймворка_noun, фреймворкое_noun, спикер_noun, ассистент_noun, платформа_noun, управление_noun, yandex_propn, cloud_propn, офлайн_noun, онлайн_noun, промокод_noun, год_noun, сайт_noun'

* __TF-IDF__

In [73]:
max(articles_df['keywords length'].to_list())

28

We can take first 30 keywords from TF-IDF because it does not provide fixed number of keywords.

In [74]:
def matrix_index(texts: list) -> csr_matrix:
    '''
    Compute inverted index in the form of a matrix.
    '''

    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), stop_words=rus_sw)
    X = vectorizer.fit_transform(texts)
    X = X.toarray()

    return vectorizer, X

In [75]:
def rank_tf_idf(index: csr_matrix, vectorizer: TfidfVectorizer):
    '''
    Get the keywords from index by sorting ngrams by TF-IDF scores.
    '''
    feature_array = np.array(vectorizer.get_feature_names_out())
    keywords = []

    for row in index:
        sorted_nums = np.argsort(row).flatten()[::-1]
        sorted_words = ', '.join(list(feature_array[sorted_nums].ravel()[:30]))
        keywords.append(sorted_words)

    return keywords

In [76]:
raw_vectorizer, raw_index = matrix_index(cleaned_texts)
tf_idf_raw = rank_tf_idf(raw_index, raw_vectorizer)

tf_idf_raw[0]

'ai, расскажет, личности, conversations, секция, расскажут, генеративных, помощника, ml, kode, онлайн, сайте conversations, разработке, ассистента, yandex cloud, yandex, промокод скидку, промокод, technology, разработке личности, виртуального помощника, виртуального, схемы, генеративных моделей, mts ai, mts, скидку, офлайн, моделей, работы'

In [77]:
lemmas_vectorizer, lemmas_index = matrix_index(lemmas[0])
tf_idf_lemmas = rank_tf_idf(lemmas_index, lemmas_vectorizer)

tf_idf_lemmas[0]

'рассказать, ai, генеративный, личность, модель, генеративный модель, секция, проектирование, conversations, разговорный, голосовой, помощник, ml, подход, инструмент, промокод, разработка личность, kode, промокод скидка, сайт conversations, банк, автоматический, yandex cloud, mts ai, mts, technology, ассистент, голосовой почта, скидка, онлайн'

In [78]:
lemmas_pos_vectorizer, lemmas_pos_index = matrix_index(lemmas[1])
tf_idf_lemmas_pos = rank_tf_idf(lemmas_pos_index, lemmas_pos_vectorizer)

tf_idf_lemmas_pos[0]

'рассказать_verb, ai_propn, личность_noun, генеративный_adj, модель_noun, conversations_propn, проектирование_noun, секция_noun, генеративный_adj модель_noun, разговорный_adj, помощник_noun, голосовой_adj, подход_noun, ml_propn, инструмент_noun, офлайн_noun, сайт_noun conversations_propn, скидка_noun, nlp_propn сервис_noun, разработка_noun личность_noun, technology_propn, голосовой_adj почта_noun, онлайн_noun, ассистент_noun, промокод_noun скидка_noun, спикер_noun, kode_propn, автоматический_adj, yandex_propn, mts_propn'

In [79]:
tf_idf_lemmas_pos_patt = []
for text in tf_idf_lemmas_pos:
    pattern_text = get_patterns(text)
    tf_idf_lemmas_pos_patt.append(pattern_text)

In [80]:
tf_idf_lemmas_pos_patt[0]

'ai_propn, личность_noun, модель_noun, conversations_propn, проектирование_noun, секция_noun, генеративный_adj модель_noun, помощник_noun, подход_noun, ml_propn, инструмент_noun, офлайн_noun, скидка_noun, разработка_noun личность_noun, technology_propn, голосовой_adj почта_noun, онлайн_noun, ассистент_noun, промокод_noun скидка_noun, спикер_noun, kode_propn, yandex_propn, mts_propn'

## Results presentation

### Raw texts

In [81]:
results_raw_df = pd.DataFrame({
    'gold tags': articles_df['gold tags'],
    'textrank': summa_raw,
    'rake trigrams': rake_raw_3,  # save only trigrams because we have trigrams in the gold standard
    'tf-idf': tf_idf_raw,
    }
)

results_raw_df

,gold tags,textrank,rake trigrams,tf-idf
0,"conversations, deeppavlov, sberdevices, yandex...","моделями, ai, генеративные модели, генеративны...","организовали управление nlp-сервисами, классич...","ai, расскажет, личности, conversations, секция..."
1,"технологии, нейронная сеть, приложение, ai, гл...","источник, источников, источники, позволяющие, ...","заключение пятнадцать опенсорс-технологий, реш...","license, ии, категория, обучения, особенности,..."
2,"1 место, хакатон, графы, альфа-банк, обработка...","данные, данных, данном, даны, признаков, призн...","использованием tf-idf представлений, последова...","пользователей, признаков, друзей, признаки, по..."
3,"rubert, сгенерировать стихотворения, lstm, ней...","своё, своих, своей, своим, шаг, шагом, это, эт...","согласно которой речь, обработке естественных ...","евгений, temperature, попробуем, онегин, the, ..."
4,"дубликаты, нейросети, entity resolution, data ...","данных, данные, данный, данной, это, эта, этих...","соревнуются state-of-the-art, работоспособном ...","zingg, entity, данных, entity resolution, запи..."


In [104]:
results_raw_df.to_csv('results_raw.tsv', sep='\t')

### Lemmas

In [82]:
results_lemmas_df = pd.DataFrame({
    'gold tags': articles_df['lemmatized gold tags'],
    'textrank': summa_lemmas,
    'rake trigrams': rake_lemmas_3,
    'tf-idf': tf_idf_lemmas
    }
)

results_lemmas_df

,gold tags,textrank,rake trigrams,tf-idf
0,"conversations, deeppavlov, sberdevices, yandex...","подход, подходить, генеративный модель рассказ...",,"рассказать, ai, генеративный, личность, модель..."
1,"технология, нейронный сеть, приложение, ai, гл...","модель, моделие, который, инструмент, изображе...","ai источник h2o, ai источник github, github","обучение, изображение, категория, license, ист..."
2,"1 место, хакатон, граф, альфа-банк, обработка ...","признак, данные, данный, пользователь, последо...","b 496d, 061266620e07n 3 0, e84b0, 9a767, 0be67...","признак, друг, хакатон, пользователь, последов..."
3,"rubert, сгенерировать стихотворение, lstm, ней...","это, евгение, евгений онегин, онегина, the, мо...","сборник произведение м, ю, как-","евгений, temperature, мочь, онегин, шаг, свой,..."
4,"дубликат, нейросеть, entity resolution, data m...","данный, zingg, который, это, любой данные, зап...","16455ntel ми core, fneendeddald4deddaгbu флешк...","zingg, запись, entity, данные, дубликат, entit..."


In [105]:
results_lemmas_df.to_csv('results_lemmas.tsv', sep='\t')

### Lemmas with POS-tags

In [83]:
results_lemmas_pos_df = pd.DataFrame({
    'gold tags': articles_df['lemmatized pos gold tags'],
    'textrank': summa_lemmas_pos,
    'rake trigrams': rake_lemmas_pos_3,
    'tf-idf': tf_idf_lemmas_pos
    }
)

results_lemmas_pos_df

,gold tags,textrank,rake trigrams,tf-idf
0,"conversations_PROPN, deeppavlov_PROPN, sberdev...","рассказать_verb, модель_noun, ai_propn, _num, ...",,"рассказать_verb, ai_propn, личность_noun, гене..."
1,"технология_NOUN, нейронный_ADJ сеть_NOUN, прил...","обучение_noun, источник_noun, _num, данные_nou...","ai_propn источник_noun h2o, ai_propn источник_...","обучение_noun, изображение_noun, категория_pro..."
2,"1_NUM место_NOUN, хакатон_NOUN, граф_NOUN, аль...","_num, признак_noun, данные_noun, пользователь_...","061266620e07n_num 3_num 0, b_propn 496d, 01000...","признак_noun, друг_noun, хакатон_noun, пользов..."
3,"rubert_PROPN, сгенерировать_VERB стихотворение...","_propn, мочь_verb, это_pron, свой_det, делать_...","_propn ю, _propn","_propn, евгений_propn, temperature_propn, мочь..."
4,"дубликат_NOUN, нейросеть_NOUN, entity_NOUN res...","_propn, _num, данные_noun, zingg_propn, которы...","first_name_propn =_x r, 16455ntel_num ми_noun ...","_propn, zingg_propn, запись_noun, entity_propn..."


In [106]:
results_lemmas_pos_df.to_csv('results_lemmas_pos.tsv', sep='\t')

### Lemmas with syntactic patterns

In [84]:
results_lemmas_pos_patt_df = pd.DataFrame({
    'gold tags': articles_df['lemmatized pos gold tags'],
    'textrank': summa_lemmas_pos_patt,
    'rake trigrams': rake_lemmas_pos_patt_3,
    'tf-idf': tf_idf_lemmas_pos_patt
    }
)

results_lemmas_pos_patt_df

,gold tags,textrank,rake trigrams,tf-idf
0,"conversations_PROPN, deeppavlov_PROPN, sberdev...","модель_noun, ai_propn, подход_noun, ml_propn, ...",,"ai_propn, личность_noun, модель_noun, conversa..."
1,"технология_NOUN, нейронный_ADJ сеть_NOUN, прил...","обучение_noun, источник_noun, данные_noun, инс...",,"обучение_noun, изображение_noun, категория_pro..."
2,"1_NUM место_NOUN, хакатон_NOUN, граф_NOUN, аль...","признак_noun, данные_noun, пользователь_noun, ...","_propn, 0be67_propn, eaaraa2aara22222222_propn...","признак_noun, друг_noun, хакатон_noun, пользов..."
3,"rubert_PROPN, сгенерировать_VERB стихотворение...","_propn, онегин_propn, онегина_propn, евгений_p...",_propn,"_propn, евгений_propn, temperature_propn, онег..."
4,"дубликат_NOUN, нейросеть_NOUN, entity_NOUN res...","_propn, данные_noun, zingg_propn, запись_noun,...",,"_propn, zingg_propn, запись_noun, entity_propn..."


In [107]:
results_lemmas_pos_patt_df.to_csv('results_lemmas_pos_patt.tsv', sep='\t')

## Evaluation

We evaluate extraction on the sets of extracted words that are in the final results.

In [91]:
def get_precision(extracted: list, gold: list) -> float:
    '''
    Compute precision score for the keywords extraction task.
    '''
    precision = len(set(extracted) & set(gold)) / len(set(extracted))

    return precision

In [92]:
def get_recall(extracted: list, gold: list) -> float:
    '''
    Compute recall score for the keywords extraction task.
    '''
    recall = len(set(extracted) & set(gold)) / len(set(gold))

    return recall

In [93]:
def get_fscore(precision: float, recall: float) -> float:
    '''
    Compute F-score score for the keywords extraction task.
    '''
    if precision == 0 or recall == 0:
        f2 = 0
    else:
        f2 = (2 * recall * precision) / (recall + precision)

    return f2

In [94]:
def get_results(extracted: list, gold: list) -> list:
    '''
    Get average scores by metrics for selected method in the list format.
    '''

    p = []
    r = []
    f = []
    for e_k, g_k in zip(extracted, gold):
        e = [ext.lower() for ext in e_k.split(', ')]
        g = [gold.lower() for gold in g_k.split(', ')]
        precision = get_precision(e, g)
        p.append(precision)
        recall = get_recall(e, g)
        r.append(recall)
        fscore = get_fscore(precision, recall)
        f.append(fscore)

    metrics = [statistics.mean(p), statistics.mean(r), statistics.mean(f)]

    return metrics

In [95]:
def create_metrics_df(results_df):
    raw_gold = results_df['gold tags'].to_list()
    metrics_raw = {'metrics': ['precision', 'recall', 'f-score']}
    for col in results_df.columns[1:]:
        raw_extracted = results_df[col].to_list()
        metrics_raw[col] = get_results(raw_extracted, raw_gold)

    metrics_df = pd.DataFrame(metrics_raw)

    return metrics_df

Raw texts:

In [96]:
metrics_raw_df = create_metrics_df(results_raw_df)

metrics_raw_df

,metrics,textrank,rake trigrams,tf-idf
0,precision,0.029319,0.030135,0.086667
1,recall,0.235374,0.438364,0.146867
2,f-score,0.047984,0.053953,0.106573


__!rake trigrams!__

In [97]:
metrics_raw_df.to_csv('metrics_raw.tsv', sep='\t')

Lemmas:

In [98]:
metrics_lemmas_df = create_metrics_df(results_lemmas_df)

metrics_lemmas_df

,metrics,textrank,rake trigrams,tf-idf
0,precision,0.035002,0.0,0.133333
1,recall,0.185428,0.0,0.220210
2,f-score,0.053752,0.0,0.162375


__!tf_idf!__

In [100]:
metrics_lemmas_df.to_csv('metrics_lemmas.tsv', sep='\t')

Lemmas with POS-tags:

In [101]:
metrics_lemmas_pos_df = create_metrics_df(results_lemmas_pos_df)

metrics_lemmas_pos_df

,metrics,textrank,rake trigrams,tf-idf
0,precision,0.044527,0.0,0.086667
1,recall,0.260736,0.0,0.141147
2,f-score,0.070713,0.0,0.105094


__!textrank!__

In [102]:
metrics_lemmas_pos_df.to_csv('metrics_lemmas_pos.tsv', sep='\t')

Lemmas with syntactic patterns:

In [103]:
metrics_lemmas_pos_patt_df = create_metrics_df(results_lemmas_pos_patt_df)

metrics_lemmas_pos_patt_df

,metrics,textrank,rake trigrams,tf-idf
0,precision,0.068348,0.0,0.128266
1,recall,0.260736,0.0,0.141147
2,f-score,0.102185,0.0,0.131035


## Discussion

* Results are not so good
* RAKE trigrams are the best on the raw texts. But I think it is because of a large number of the extracted keywords for each text.
* Quality of TextRank is overall good.
* Syntactic patterns give better quality when working with tags.
* Needed keywords are often parts of collocations.

__TODO:__
* Fix omissions in the data after parsing (_propn after TextRank and TF-IDF).
* Morphological and syntactical parsing of the gold standard?
* There is a little problem with markup of the POS-tags because abbreviations in our case have tags PROPN and NOUN but parser put PROPN tag on all abbreviations. So by metrics the results must be better.
* Threshold for the output of models.
* Lemmas comparison for the raw texts.
* Reward for partial matching.

## Conclusions

* On the Habr the authors are those who tag their articles. So the unite algorithm to put tags does not exist.
* From the syntactical point of view users prefer different NP`s. They do not prefer to put specific names in the tags but they mention a lot of abbreviations and common domain words. From the semantic point of view the also prefer put abstract common domain expressions that can not refer to the specific expressions in the text.
* RAKE extractor can not work with lemmatized texts. `summa` has problems with tagged words.
* From metrics results we can see that models solve keyword extraction task not so good. 
* I think that by quality the work of models is quite good for machine keyword extraction because they catched a lot of informative words.